## Week 06

You can also run `word_count_submit.py` with  
```bash
pyspark-submit word_count_submit.py
```
in Anaconda Prompt (miniconda3)

In this week, we want to compare how fast is pyspark to process eight books.

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
import numpy as np

import glob
import pandas as pd
import re

from collections import Counter

In [3]:
spark = (SparkSession
  .builder
  .master("local[*]")   # optional
  .appName("Word Counts Program.")
  .getOrCreate())

A single novel (Jane Austen 1813 - Pride and Prejudice)

In [4]:
# Computational times: (10.3 secs first run, 0.2 - 0.4 secs second runs)
results = (
  spark.read.text("./data/gutenberg_books/1342-0.txt")
  .select(F.split(F.col("value"), " ").alias("line"))
  .select(F.explode(F.col("line")).alias("word"))
  .select(F.lower(F.col("word")).alias("word"))
  .select(F.regexp_extract(F.col("word"), "[a-z]+", 0).alias("word"))
  .where(F.col("word") != "")
  .groupby("word")
  .count()
)

# Show the top 10 of the most occurrence words in Jane Austen - Pride and Prejudice
results.orderBy("count", ascending=False).show(10)

+----+-----+
|word|count|
+----+-----+
| the| 4496|
|  to| 4235|
|  of| 3719|
| and| 3602|
| her| 2223|
|   i| 2052|
|   a| 1997|
|  in| 1920|
| was| 1844|
| she| 1703|
+----+-----+
only showing top 10 rows



8 classical books
- `11-0.txt`: Lewis Carol (1865) - Alice's Adventures in Wonderland
- `84-0.txt`: Mary Shelley (1818) - Frankenstein; or, The Modern Promotheus
- `1342-0.txt`: Jane Austen (1813) - Pride and Prejudice
- `1661-0.txt`: Arthur Conan Doyle (1892) - The Adventures of Sherlock Holmes
- `2701-0.txt`: Herman Melville (1851) - Moby-Dick; or, The Whale
- `pg132.txt`: 孫子/Sun Tzu (5th century BC) - 孫子兵法 (The Art of War / Sun Tzu's Military Method) 
- `pg514.txt`: Louisa May Alcott (1868-1869) - Little Women
- `pg1399.txt`: Лев Толстой/Leo Tolstoy (1878) - Анна Каренина (Anna Karenina)

In [5]:
# Computational time: 5.1 secs for first run; 0.5 - 0.7 for the second runs
results = (
  spark.read.text("./data/gutenberg_books/*.txt")
  .select(F.split(F.col("value"), " ").alias("line"))
  .select(F.explode(F.col("line")).alias("word"))
  .select(F.lower(F.col("word")).alias("word"))
  .select(F.regexp_extract(F.col("word"), "[a-z]+", 0).alias("word"))
  .where(F.col("word") != "")
  .groupby("word")
  .count()
)

# Show the top 10 of the most occurrence words in all books inside `data/project-gutenberg/`
results.orderBy("count", ascending=False).show(10)

+----+-----+
|word|count|
+----+-----+
| the|60331|
| and|39571|
|  to|31793|
|  of|30994|
|   a|23322|
|  in|19247|
|   i|19189|
|that|15784|
|  he|15253|
|  it|14744|
+----+-----+
only showing top 10 rows



Let us compare to `pandas` implementation.   
It takes so long to calculate compare to when using `pyspark`.   
`pandas` uses __eager evaluation__ and  
`pyspark` uses __lazy evaluation__

In [6]:
files = glob.glob("./data/gutenberg_books/*.txt")

lines = []
for file in files:
  with open(file, 'r') as f:
    lines += f.readlines()

results = pd.DataFrame(lines, columns=["value"])
results["value"] = results["value"].str.split(" ")
results = results.explode("value")
results.rename(columns={"value": "word"}, inplace=True)
results["word"] = results["word"].str.lower()
results["word"] = results["word"].str.extract(r"([a-z]+)")

# results.head(30)
# results[results["word"].isna() == True]
results.dropna(inplace=True)
# results.reset_index(inplace=True, drop=True)
results["count"] = 1
results = results.groupby(by=["word"], as_index=False, sort=False).count()
results.sort_values(by=["count"], inplace=True, ascending=False)
results.head(10)

,word,count
13,the,60331
21,and,39571
66,to,31793
15,of,30994
90,a,23322
4,in,19247
61,i,19189
127,that,15784
686,he,15253
29,it,14744


### Program Broadcast

Download the dataset from the Google Drive with you ITK's account:
https://drive.google.com/drive/u/0/folders/1-PndwIh7saR0ZocE3Ujslx8vei-HU9k8

In [7]:
spark = (SparkSession
  .builder
  .master("local[*]")   # optional
  .appName("Processing Tabular Data")
  .getOrCreate())

Read broadcast data

In [11]:
# DIRECTORY = "./data/broadcast_logs"   # use this when you present to the student
DIRECTORY = "../rioux-2022/data/broadcast_logs"

broadcast_logs_filename = "BroadcastLogs_2018_Q3_M8_sample.CSV"
# broadcast_logs_filename = "BroadcastLogs_2018_Q3_M8.CSV"

logs = spark.read.csv(
  os.path.join(DIRECTORY, broadcast_logs_filename),
  sep="|",
  header=True,
  inferSchema=True,
  timestampFormat="yyyy-MM-dd")

In [12]:
logs.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

In [13]:
logs.show(10)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+
|BroadcastLogID|LogServiceID|   LogDate|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|        Duration|         EndTime|LogEntryDate|ProductionNO|        ProgramTitle|       StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|
+--------------+------------+----------+----------+-------------------

Accessing all columns names

In [14]:
logs.columns

['BroadcastLogID',
 'LogServiceID',
 'LogDate',
 'SequenceNO',
 'AudienceTargetAgeID',
 'AudienceTargetEthnicID',
 'CategoryID',
 'ClosedCaptionID',
 'CountryOfOriginID',
 'DubDramaCreditID',
 'EthnicProgramID',
 'ProductionSourceID',
 'ProgramClassID',
 'FilmClassificationID',
 'ExhibitionID',
 'Duration',
 'EndTime',
 'LogEntryDate',
 'ProductionNO',
 'ProgramTitle',
 'StartTime',
 'Subtitle',
 'NetworkAffiliationID',
 'SpecialAttentionID',
 'BroadcastOriginPointID',
 'CompositionID',
 'Producer1',
 'Producer2',
 'Language1',
 'Language2']

Print the table in a group of three columns.    
First we split array column into three items on each group

In [15]:
column_split = np.array_split(
  np.array(logs.columns), len(logs.columns) // 3)
print(column_split)

[array(['BroadcastLogID', 'LogServiceID', 'LogDate'], dtype='<U22'), array(['SequenceNO', 'AudienceTargetAgeID', 'AudienceTargetEthnicID'],
      dtype='<U22'), array(['CategoryID', 'ClosedCaptionID', 'CountryOfOriginID'], dtype='<U22'), array(['DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID'],
      dtype='<U22'), array(['ProgramClassID', 'FilmClassificationID', 'ExhibitionID'],
      dtype='<U22'), array(['Duration', 'EndTime', 'LogEntryDate'], dtype='<U22'), array(['ProductionNO', 'ProgramTitle', 'StartTime'], dtype='<U22'), array(['Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID'],
      dtype='<U22'), array(['BroadcastOriginPointID', 'CompositionID', 'Producer1'],
      dtype='<U22'), array(['Producer2', 'Language1', 'Language2'], dtype='<U22')]


In [16]:
for x in column_split:
  logs.select(*x).show(5, False)

+--------------+------------+----------+
|BroadcastLogID|LogServiceID|LogDate   |
+--------------+------------+----------+
|1196192316    |3157        |2018-08-01|
|1196192317    |3157        |2018-08-01|
|1196192318    |3157        |2018-08-01|
|1196192319    |3157        |2018-08-01|
|1196192320    |3157        |2018-08-01|
+--------------+------------+----------+
only showing top 5 rows

+----------+-------------------+----------------------+
|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|
+----------+-------------------+----------------------+
|1         |4                  |NULL                  |
|2         |NULL               |NULL                  |
|3         |NULL               |NULL                  |
|4         |NULL               |NULL                  |
|5         |NULL               |NULL                  |
+----------+-------------------+----------------------+
only showing top 5 rows

+----------+---------------+-----------------+
|CategoryID|ClosedCaptionID|Co

Remove column using `.drop()`

In [17]:
logs_clean = logs.drop("BroadcastLogID", "SequenceNO")

# Testing if we effectively got rid of the columns
print("BroadcastLogID" in logs_clean.columns)
print("SequenceNo" in logs_clean.columns)

False
False


View `Duration` column

In [18]:
logs_clean.select(F.col("Duration")).show(5)

print(logs_clean.select(F.col("Duration")).dtypes)

+----------------+
|        Duration|
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows

[('Duration', 'string')]


Parse hour, minute and second for each row in `logs_clean`

In [19]:
logs_clean.select(
  F.col("Duration"),
  F.col("Duration").substr(1, 2).cast("int").alias("dur_hours"),
  F.col("Duration").substr(4, 2).cast("int").alias("dur_minutes"),
  F.col("Duration").substr(7, 2).cast("int").alias("dur_seconds"),
).distinct().show(5)

+----------------+---------+-----------+-----------+
|        Duration|dur_hours|dur_minutes|dur_seconds|
+----------------+---------+-----------+-----------+
|00:04:52.0000000|        0|          4|         52|
|00:10:06.0000000|        0|         10|          6|
|00:09:52.0000000|        0|          9|         52|
|00:04:26.0000000|        0|          4|         26|
|00:14:59.0000000|        0|         14|         59|
+----------------+---------+-----------+-----------+
only showing top 5 rows



We will discuss next week for getting the above time parsing columns